In [ ]:
# General dependencies
import datetime as dt
import pandas as pd

# APIs dependencies
import json
import requests
from pprint import pprint
from resources.api_keys import owm_api_key as owm_key

# Mapping dependencies
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt

# API endpoint
url_hstr = 'https://api.openweathermap.org/data/2.5/uvi/history?'
url_fcst = 'https://api.openweathermap.org/data/2.5/onecall?'
units = 'metric'

In [ ]:
# Generate grid with timestamps 8 days
def grid_dte_pnts():
    
    # Empty list to grid locations
    grid_list = []


    # Generate list of grid pairs and timestamps
    
    # Western Australia
    # latitude range -36 to -12 deg south
    # longitude range 110 to 130 deg east

    lat_rng = [-36, -12]
    lon_rng = [112, 130]
    dte_stt = int(dt.datetime.strptime('2020-01-13 12:00:00 UTC', '%Y-%m-%d %H:%M:%S %Z').timestamp())
    dte_cnt = 8
      
       
    for dte in range(dte_stt, dte_stt + dte_cnt*(86400), 86400):
        for lat in range(lat_rng[0], lat_rng[1] + 1, 4):
            for lon in range(lon_rng[0], lon_rng[1] + 1, 4):

                grid_pt = [lat, lon, dte]

                # Append to list
                grid_list.append(grid_pt)

    return grid_list


# Partial query URL
qery_url_hstr = f'{url_hstr}&appid={owm_key}'

# Empty variables to hold response
hstr_data = {}
hstr_data['date'] = []
hstr_data['latitude'] = []
hstr_data['longitude'] = []
hstr_data['uv-index'] = []

# uv index - historical data
# Loop through the grid and perform a request for data on each
# Call function 'grid_pnts' to provide coordinates
for locn in grid_dte_pnts():  

    print(f'... {locn} ...')
    
    # If location in OWM database then collect data
    try:
        
        # request uv index data for location
        response = requests.get(qery_url_hstr + '&lat=' + str(locn[0]) + '&lon=' + str(locn[1])  + '&start=' + str(locn[2]) + '&end=' + str(locn[2])).json()
        
        # append to json string
        hstr_data['date'].append(dt.datetime.utcfromtimestamp(response[0]['date']).strftime('%Y-%m-%d %H:%M'))
        hstr_data['latitude'].append(response[0]['lat'])
        hstr_data['longitude'].append(response[0]['lon'])
        hstr_data['uv-index'].append(response[0]['value'])

    # If location not in OWM database then report to log and skip to next location
    except KeyError:
        print(f'  {locn} not found ...skipping...')

# write json to file
with open('analysis/hstr_data.json', 'w') as outfile:
    json.dump(hstr_data, outfile)        

# print to terminal
pprint(hstr_data)
print(f'  Finished data collection')

In [ ]:
# Generate grid 
def grid_pnts():
    
    # Empty list to grid locations
    grid_list = []


    # Generate list of grid pairs 
    
    # Australia
    # latitude range -45 to -8 deg south
    # longitude range 110 to 156 deg east

    lat_rng = [-36, -12]
    lon_rng = [112, 130]
     
       
    for lat in range(lat_rng[0], lat_rng[1] + 1, 4):
        for lon in range(lon_rng[0], lon_rng[1] + 1, 4):

            grid_pt = [lat, lon]

            # Append to list
            grid_list.append(grid_pt)

    return grid_list


# Partial query URL
qery_url_fcst = f'{url_fcst}&appid={owm_key}&exclude=minutely,hourly,alerts&units={units}'

# Empty variables to hold response
fcst_data = {}
fcst_data['date'] = []
fcst_data['latitude'] = []
fcst_data['longitude'] = []
fcst_data['uv-index'] = [] 
fcst_data['max-temperature'] = [] 

# uv and temp forecast
# Loop through the grid and perform a request for data on each
# Call function 'grid_pnts' to provide coordinates
for locn in grid_pnts():  

    print(f'... {locn} ...')
    
    # If location in OWM database then collect data
    try:
        
        # request uv and temperature data for location
        response = requests.get(qery_url_fcst + '&lat=' + str(locn[0]) + '&lon=' + str(locn[1])).json()
        
        # append forecasts to json string
        for fcst in response['daily']:
            
            # append to json string
            fcst_data['date'].append(dt.datetime.utcfromtimestamp(fcst['dt']).strftime('%Y-%m-%d %H:%M'))
            fcst_data['latitude'].append(response['lat'])
            fcst_data['longitude'].append(response['lon'])
            fcst_data['uv-index'].append(fcst['uvi'])
            fcst_data['max-temperature'].append(fcst['temp']['max'])

    # If location not in OWM database then report to log and skip to next location
    except KeyError:
        print(f'  {locn} not found ...skipping...')

# write json to file
with open('analysis/fcst_data.json', 'w') as outfile:
    json.dump(fcst_data, outfile)
        
# print to terminal
pprint(fcst_data)
print(f'  Finished data collection')